In [29]:
import numpy as np
import matplotlib.pyplot as plt
import json

In [ ]:
collection=[
    {
        "in_f":"C:\\Users\\Pragiya\\MSC_Research\\LVLane\\my_anno_all\\my_combined_key_frames\\20\\refined_20-40.json",
        "out_f":"C:\\Users\\Pragiya\\MSC_Research\\LVLane\\my_anno_all\\my_combined_key_frames\\20\\prerefined_20-40.json",
        "raw":"20"
    }
]
for person in collection:
    print("folder name::",person["raw"])
    fileInitialFormatter(person["in_f"],person["out_f"],person["raw"])

In [44]:

for raw in range(160,180,20):
    #raw=i*10
    in_f=f"C:\\Users\\Pragiya\\MSC_Research\\LVLane\\my_anno_all\\my_combined_key_frames\\{raw}\\refined_{raw}-{raw+20}.json"
    out_f=f"C:\\Users\\Pragiya\\MSC_Research\\LVLane\\my_anno_all\\my_combined_key_frames\\{raw}\\prerefined_{raw}-{raw+20}.json"
    print(raw,"\n",in_f,"\n",out_f)
    try:
        fileInitialFormatter(in_f,out_f,f"raw")
    except Exception as e:
        print(f"Error caught: {e}, but continuing the loop.")
        print("except::",raw,"\n",in_f,"\n",out_f)


160 
 C:\Users\Pragiya\MSC_Research\LVLane\my_anno_all\my_combined_key_frames\160\refined_160-180.json 
 C:\Users\Pragiya\MSC_Research\LVLane\my_anno_all\my_combined_key_frames\160\prerefined_160-180.json
1.jpg
1.jpg
1.jpg
2.jpg
2.jpg
2.jpg
3.jpg
3.jpg
3.jpg
4.jpg
4.jpg
4.jpg
5.jpg
5.jpg
5.jpg
6.jpg
6.jpg
6.jpg
7.jpg
7.jpg
7.jpg
8.jpg
8.jpg
8.jpg
9.jpg
9.jpg
9.jpg
10.jpg
10.jpg
10.jpg
11.jpg
11.jpg
11.jpg
12.jpg
12.jpg
12.jpg
13.jpg
13.jpg
13.jpg
14.jpg
14.jpg
14.jpg
15.jpg
15.jpg
15.jpg
16.jpg
16.jpg
16.jpg
17.jpg
17.jpg
17.jpg
18.jpg
18.jpg
18.jpg
19.jpg
19.jpg
19.jpg
20.jpg
20.jpg
20.jpg


In [ ]:
for i in range(10, 200, 20):
    try:
        print(f"Loop {i}")
        if i == 90:  # Simulating an error at i = 90
            raise ValueError("Something went wrong at 90!")
    except ValueError as e:
        print(f"Error caught: {e}, but continuing the loop.")

print("Loop completed successfully!")


In [ ]:
def fileInitialFormatter(in_filename,out_fileName,rawName):
    # Opening JSON file
    f = open(in_filename)
    
    # returns JSON object as 
    # a dictionary
    data = json.load(f)
    #data

    # create tusimple format labels from via labels using spline
    from scipy.interpolate import CubicSpline


    h_samples = []
    for i in range(160,720,10):
        h_samples.append(i)

    with open(out_fileName, "w") as outfile:
        for each_image in data:
            label = data[each_image]
            filename = label["filename"]
            lanes = np.array([], dtype=int)
            count_lanes = 0
            data_samples = []
            for each_lane in label["regions"]:           
                label_coordinates = each_lane["shape_attributes"]
                x = label_coordinates["all_points_x"]
                y = label_coordinates["all_points_y"]
                data_samples = [(slp, cls) for slp, cls in zip(x, y)]
                data_samples.sort(key = lambda x: x[1]) 
                x_cor = [] 
                y_cor = []
                for sample in data_samples:
                    x_cor.append(sample[0])
                    y_cor.append(sample[1])
                    
                y_values = [i//10 for i in y_cor]
                cs = CubicSpline(y_cor, x_cor)
                y_min = min(y_values)*10+10

                tusimple_y = []
                tusimple_x = []
                for i in range(y_min, max(y_values)*10, 10):
                    tusimple_y.append(i)
                    tusimple_x.append(int(cs(i)))
                print(filename)
                y_min = tusimple_y[0]/10
                y_max = tusimple_y[-1]/10
                front = int(y_min - 16)
                back = int(71- y_max)
                front_arr = np.ones(front, dtype=int)*(-2)
                back_arr = np.ones(back, dtype=int)*(-2)
                tusimple_x = np.array([np.hstack([front_arr, tusimple_x, back_arr])])
                
                count_lanes += 1
                if count_lanes >1:
                    lanes = np.vstack([lanes, tusimple_x])
                else:
                    lanes = tusimple_x
                #print(lanes)

            dictionary = {"lanes": lanes.tolist(),
                    "h_samples": h_samples,
                    "raw_file": "clips/"+rawName+filename
            }
            
            # Serializing json
            json_object = json.dumps(dictionary)
            outfile.write(json_object)
            outfile.write('\n')
            

In [45]:

for raw in range(160,180,20):
    #raw=i*10
    in_f=f"C:\\Users\\Pragiya\\MSC_Research\\LVLane\\my_anno_all\\my_combined_key_frames\\{raw}\\prerefined_{raw}-{raw+20}.json"
    out_f=f"C:\\Users\\Pragiya\\MSC_Research\\LVLane\\my_anno_all\\my_combined_key_frames\\{raw}\\orefined_{raw}-{raw+20}.json"
    print(raw,"\n",in_f,"\n",out_f)
    try:
        coodinateCaliberate(in_f,out_f,f"raw")
    except Exception as e:
        print(f"Error caught: {e}, but continuing the loop.")
        print("except::",raw,"\n",in_f,"\n",out_f)

160 
 C:\Users\Pragiya\MSC_Research\LVLane\my_anno_all\my_combined_key_frames\160\prerefined_160-180.json 
 C:\Users\Pragiya\MSC_Research\LVLane\my_anno_all\my_combined_key_frames\160\orefined_160-180.json


In [20]:
# order lanes from left to right using slope, also delete any lanes with less than 4 coordinates
def coodinateCaliberate(in_f,out_f,raw):

    with open(out_f, 'w') as outfile:
        json_path = in_f
        with open(json_path) as f:
            json_lines = f.readlines()
            line_index = 0
            while line_index < len(json_lines):
                line = json_lines[line_index]
                label = json.loads(line)

                # ---------- clean and sort lanes -------------
                lanes = []
                _lanes = []
                slope = [] # identify 0th, 1st, 2nd, 3rd, 4th, 5th lane through slope
                for i in range(len(label['lanes'])):
                    l = [(x, y) for x, y in zip(label['lanes'][i], label['h_samples']) if x >= 0]
                    if (len(l)>1):
                        _lanes.append(l)
                        slope.append(np.arctan2(l[-1][1]-l[0][1], l[0][0]-l[-1][0]) / np.pi * 180)
                _lanes = [_lanes[i] for i in np.argsort(slope)]   # arrange lanes based on slope
                data = [(ind, slp) for ind, slp in enumerate(slope)]
                data.sort(key = lambda x: x[1])                   # arrange (slope, class_list) based on slope
                slope = [slope[i] for i in np.argsort(slope)]     # arrange slope low to high

                ind = [c[0] for c in data]
                label['lanes'] = [label['lanes'][i] for i in ind]
                
                x = []
                for i in range(len(label['lanes'])):
                    l = [(x, y) for x, y in zip(label['lanes'][i], label['h_samples']) if x >= 0]
                    if len(l) < 4:
                        x.append(i)
                for index in sorted(x, reverse=True):
                    del label['lanes'][index]
                if len(label['lanes']) > 0:
                    json_object = json.dumps(label)
                    outfile.write(json_object)
                    outfile.write('\n')

                line_index += 1